In [10]:
#Import the create_engine() function
from sqlalchemy import create_engine
from sqlalchemy import update
import pymysql
import pandas as pd
import numpy as np
from numpy import inf
##Create the Engine Class 
#The engine class connects a pool and dialect together to provide a source of 
#dB connectivity and behavior 

#The create_engine() function takes the db as one arg. 
engine = create_engine(
        "mysql+pymysql://root:abcDEFhij123!@@localhost/coffee_database", 
        echo = True
        )
#The echo flag is a shortcut to set up SQL Alchemy Logging. 
#Every time we execute an expression, the console will outpute 
#the corresponding SQL expression 



##Executing Textual Expressions 
from sqlalchemy import text
#Obtain a Connection Object representing an actively checked out DBAPI 
#connection resource 
conn = engine.connect()

txt = text("SELECT * FROM user")
result = conn.execute(txt)
result.close()
users=pd.read_sql('select * from user left join orders on user_id=user_id_o left join coffee_order on order_id_co=order_id left join coffee_products on coffee_id=coffee_id_co left join aroma_coffee on coffee_id_a=coffee_id left join aroma on aroma.flavor_aroma_id=aroma_coffee.flavor_aroma_id',"mysql+pymysql://root:abcDEFhij123!@@localhost/coffee_database")



2019-07-28 15:47:13,915 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2019-07-28 15:47:13,918 INFO sqlalchemy.engine.base.Engine {}
2019-07-28 15:47:13,925 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2019-07-28 15:47:13,926 INFO sqlalchemy.engine.base.Engine {}
2019-07-28 15:47:13,932 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2019-07-28 15:47:13,933 INFO sqlalchemy.engine.base.Engine {}
2019-07-28 15:47:13,935 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2019-07-28 15:47:13,936 INFO sqlalchemy.engine.base.Engine {}
2019-07-28 15:47:13,944 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2019-07-28 15:47:13,945 INFO sqlalchemy.engine.base.Engine {}
2019-07-28 15:47:13,947 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2019-07-28 15:47:13,949 INFO sqlalchemy.engine.base.E

In [11]:

rate2=pd.read_sql('select user_id, coffee_id from user join coffee_products left join rating on (rating.reviewer_id=user_id and rating.coffee_id_r=coffee_id) group by user_id, coffee_id order by user_id desc',"mysql+pymysql://root:abcDEFhij123!@@localhost/coffee_database")
rate=pd.read_sql('select user_id, coffee_id,sum(rating) as total_rating from user join coffee_products left join rating on (rating.reviewer_id=user_id and rating.coffee_id_r=coffee_id) group by user_id, coffee_id order by user_id desc',"mysql+pymysql://root:abcDEFhij123!@@localhost/coffee_database")

In [12]:
R=rate.pivot_table(index="user_id",columns="coffee_id",values="total_rating",dropna=False, fill_value=0)

In [4]:
rate2.shape
import random 
rating_choices = [1,2,3,4,5]
rating_choices_sample = [random.choice(rating_choices) for i in range(rate2.shape[0])]
rate2['total_rating']=rating_choices_sample
rate2.head()
rate2.to_csv("review.csv")

In [13]:
import numpy as np
normR=np.linalg.norm(np.array(R))

In [14]:
aR=np.array(R)
tranaR=np.transpose(aR)

In [15]:
Q=np.diag(np.diag(tranaR@aR))

In [16]:
Qstar=np.diag(np.diag(np.power(Q,-.5)))

C:\Users\noahd\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in power
  """Entry point for launching an IPython kernel.


In [17]:

Qstar[Qstar == inf]=0
Qstar

array([[0.0060804 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.00606919, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.00605905, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.00599028, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.00605527,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.0061665 , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00609983, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.       

In [18]:
P=aR@tranaR

In [19]:
Pstar=np.diag(np.diag(np.power(P,-.5)))

In [20]:
Pstar[Pstar==inf]=0

In [21]:
predictedRatingUser=Pstar@P@Pstar@aR
predictedRatingCoffee=aR@Qstar@Q@Qstar

In [22]:
pRU=pd.DataFrame(predictedRatingUser,columns=R.columns)

In [23]:
R.update(pRU)

In [24]:
R['user_id']=R.index

In [25]:
Rback=pd.melt(R,id_vars='user_id',var_name='title',value_name='target_rating')

In [26]:
Rback.sort_values(by=['user_id'],ascending=False).reset_index(drop=True)

,user_id,title,target_rating
0,2480,10,1.000000
1,2480,8,2.000000
2,2480,5,2.000000
3,2480,3,4.000000
4,2480,6,2.000000
5,2480,2,2.000000
6,2480,4,5.000000
7,2480,7,3.000000
8,2480,9,1.000000
9,2480,1,3.000000


In [27]:
 Rback['target_rating'] = Rback.groupby(['user_id'])['target_rating'].rank(ascending=1)


In [28]:
Rback.to_csv('recommendation.csv')

In [30]:
sum(Rback['target_rating'].value_counts())

24800